# PyMapD and PyGDF Demo on NY Taxi Data Subset

In this example, we will use PyMapD to create and populate a table of NY Taxi data from a CSV file.  Then, we query the MapD database to get a PyGDF GPU dataframe and manipulate the data using groupby, join and other dataframe operations.

In [1]:
import cudf
import cuml

### Load data from csv

Decompress CSV archive

In [ ]:
!gunzip -kf ./data/nytaxi/nytaxi_pre_mapd_200k.csv.gz

## Connect to MapD

In [51]:
df = cudf.read_csv("data/nytaxi/nytaxi_pre_mapd_200k.csv")
#print(df.dtypes)
dnames = list(df)
types = []
for i in range (len(list(df))):
    tdtype = str(df.dtypes[i])
    if(tdtype == 'int32'):
        tdtype = 'str'
    elif(tdtype == 'int64'):
        tdtype = 'int'
    elif(tdtype == 'datetime64[ms]'):
        tdtype = 'date'
    types.append(tdtype)

print(dnames)
print(types)
df2 = cudf.io.csv.read_csv_strings('data/nytaxi/nytaxi_pre_mapd_200k.csv', delimiter=',',
                                       names = dnames,
                                       dtype= types,
                                       quoting=True,
                                       doublequote=False,
                                       skiprows=1)
print(df2)

['vendor_id', 'rate_code', 'store_and_fwd_flag', 'passenger_count', 'trip_time_in_secs', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'tolls_amount', 'tip_amount', 'total_amount', 'mta_tax', 'fare_amount', 'payment_type', 'surcharge', 'pickup_datetime_year', 'pickup_datetime_month', 'pickup_datetime_day', 'pickup_datetime_hours', 'dropoff_datetime_year', 'dropoff_datetime_month', 'dropoff_datetime_day', 'dropoff_datetime_hours']
['str', 'int', 'str', 'int', 'int', 'float64', 'float64', 'float64', 'date', 'date', 'float64', 'float64', 'float64', 'float64', 'float64', 'str', 'float64', 'int', 'int', 'int', 'int', 'int', 'int', 'int', 'int']
[<nvstrings count=200000>, <cudf.Series nrows=200000 >, <nvstrings count=200000>, <cudf.Series nrows=200000 >, <cudf.Series nrows=200000 >, <cudf.Series nrows=200000 >, <cudf.Series nrows=200000 >, <cudf.Series nrows=200000 >, <cudf.Series nrows=200000 >, <cudf.Series nrows=200000 >, <cudf.Series nro

In [53]:
print(df2.head(5))

AttributeError: 'list' object has no attribute 'head'

Inspect column types

In [7]:
df.dtypes

NameError: name 'df' is not defined

### Make table

In [ ]:
print(df.head(5))

In [ ]:
print(df["payment_type"], df["vendor_id"])

In [ ]:
df.to_csv('foo3.csv', index=False)

In [ ]:
print('nrows', len(df))

## Groupby lat lon grid

We want to group each record by their pickup location. We will to round the lat lon with the ``round_latlon`` method.  By using ``.applymap``, the rounding method will be compiled into GPU code.

In [ ]:
from math import floor
def round_latlon(x):
    scale = 5
    return floor(x * scale) / scale

In [ ]:
group_df = df.loc[:, ['pickup_longitude', 'pickup_latitude', 'tip_amount', 'fare_amount']] 
print(df['tip_amount'])
group_df['pickup_longitude'] = group_df['pickup_longitude'].applymap(round_latlon)
group_df['pickup_latitude'] = group_df['pickup_latitude'].applymap(round_latlon)

group_df['tip_ratio'] = group_df['tip_amount'] / group_df['fare_amount']
print(group_df['tip_ratio'], group_df['tip_amount'])


In [ ]:
group_df.dtypes

In [ ]:
print('group df rows', len(group_df))
print(group_df.head(5))

Here, we run groupby and specify the aggregating methods on each column.

In [ ]:
from numba import cuda,jit,float32

# Aggregating methods to apply to each column

aggs = {
    'tip_amount': ['mean'],
    'fare_amount': ['mean', 'count'], ##std is not currently impletmented in RAPIDS 0.5.1.  Therefore, we need a work around (thanks Jiwei Liu!)
    'tip_ratio': ['mean']
}

grouped_stats = group_df.groupby(['pickup_longitude', 'pickup_latitude']).agg(aggs)
grouped_std = [group_df['fare_amount'].std()]*len(grouped_stats) ### STD is a SPSA method, while the other aggs are SPMA.  This requires us to calculate it seperatedly, put the values in an series  of ismilar size, and then join it to the dataframe
grouped_stats['std_fare_amount'] = grouped_std
print('total groups', len(grouped_stats))
print(grouped_stats.head())
print(grouped_stats)

Reorder the grouped dataframe by `count_fare_amount`

In [ ]:
print(grouped_stats.sort_values('count_fare_amount', ascending=False))


## Groupby payment type

We can also group by categorical columns.

In [ ]:
group_pay = df.loc[:, ['payment_type', 'tip_amount', 'fare_amount']]
print(group_pay)
group_pay['tip_ratio'] = group_df['tip_ratio']
print(group_df['tip_ratio'])
groupby_payment = group_pay.groupby(['payment_type']).mean()
display(groupby_payment.sort_values('mean_tip_ratio', ascending=False))

### Load data from csv

Decompress CSV archive

In [ ]:
!gunzip -kf ./data/nytaxi/nytaxi_pre_mapd_200k.csv.gz

Load data

## Ingest data from MapD

In [ ]:
df = con.select_ipc_gpu("""
SELECT  
payment_type, pickup_longitude, pickup_latitude, tip_amount, total_amount, fare_amount
FROM nytaxi_subset 
""")

Check the type of the result from `con.select_ipc_gpu` is a GPU dataframe

In [ ]:
type(df)

In [ ]:
print('nrows', len(df))

Inspect column types

In [ ]:
df.dtypes

In [ ]:
df.head().to_pandas()

In [ ]:
from math import floor

def round_latlon(x):
    scale = 5
    return floor(x * scale) / scale

In [ ]:
group_df = df.loc[:, ['pickup_longitude', 'pickup_latitude', 'tip_amount', 'fare_amount']] 

group_df['pickup_longitude'] = group_df['pickup_longitude'].applymap(round_latlon)
group_df['pickup_latitude'] = group_df['pickup_latitude'].applymap(round_latlon)

group_df['tip_ratio'] = group_df['tip_amount'] / group_df['fare_amount']


In [ ]:
group_df.dtypes

In [ ]:
group_df.head().to_pandas()

Here, we run groupby and specify the aggregating methods on each column.

In [ ]:
from collections import OrderedDict

# Aggregating methods to apply to each column
grouped_std = group_df['fare_amount'].std()
aggs = {
    'tip_amount': ['mean'],
    'fare_amount': ['mean', 'count'], ##std is not currently impletmented in RAPIDS 0.5.1.  Therefore, we need a work around (thanks Jiwei Liu!)
    'tip_ratio': ['mean']
}
print(aggs)
print(len(aggs))

#print(group_df.groupby(['pickup_longitude', 'pickup_latitude']).agg(aggs))
grouped_stats = group_df.groupby(['pickup_longitude', 'pickup_latitude']).agg(aggs)
#grouped_stats = group_stats.assign(std_fare_amount=grouped_std)
for i in range(len(grouped_stats)):
    grouped_stats[i].std_fare_amount= grouped_std

print('total groups', len(grouped_stats))
grouped_stats.head().to_pandas()

Reorder the grouped dataframe by `fare_amount_count`

In [ ]:
grouped_stats.sort_values('fare_amount_count', ascending=False).head().to_pandas()

## Groupby payment type

We can also group by categorical columns.

In [ ]:
group_pay = df.loc[:, ['payment_type', 'tip_amount', 'fare_amount']]
group_pay['tip_ratio'] = group_df['tip_ratio']

groupby_payment = group_pay.groupby(['payment_type']).mean()
groupby_payment.sort_values('tip_ratio', ascending=False).to_pandas()

## Join table with payment_type meaning

We can use `.join()` to add a description column for each payment type

In [ ]:
import pandas
import numpy as np

payment_code = {
    'CRD': 'Credit Card',
    'CSH': 'Cash',
    'NOC': 'No Charge',
    'DIS': 'Dispute',
    'UNK': 'Unknown',
}

payment_meaning = pygdf.DataFrame()

# Customize codes.dtype to match storage type from mapd
src_cat = group_pay.payment_type
cat = pandas.Categorical(payment_code.keys(), categories=src_cat.cat.categories)
payment_meaning['payment_type'] = pygdf.Series.from_categorical(cat, codes=cat.codes.astype(src_cat.data.dtype))

payment_meaning['payment_meaning'] = pandas.Categorical(payment_code.values())
payment_meaning = payment_meaning.set_index('payment_type')

payment_meaning.to_pandas()

In [ ]:
joined = groupby_payment.set_index('payment_type').join(payment_meaning)
joined.sort_values('tip_ratio', ascending=False).to_pandas()